In [1]:
import argparse
from datetime import datetime
import json
import numpy as np
import operator
import os
import pandas as pd
from tqdm.notebook import tqdm

In [62]:
market = 'NASDAQ'
data_path_eod = '/home/kikyo/data/qt/google_finance/'
data_path_attn = '/home/kikyo/data/qt/attention_data_' + market + '/'
hpg = np.load('/home/kikyo/code/hidden-conc-hypg/hg_' + market + '.npy')
data_path_rr = '/home/kikyo/data/qt/rr8_volumn_' + market + '/'
date_format = '%Y-%m-%d %H:%M:%S'
tickers = '/home/kikyo/data/qt/' + market + '_tickers_qualify_dr-0.98_min-5_smooth.csv'
tickers = np.genfromtxt(tickers, dtype=str, delimiter='\t', skip_header=False)
begin_time = datetime.strptime('2012-11-19 00:00:00', date_format)
col = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'rr-1', 'rr-5', 'rr-30', 'volumn_ratio', 'volumn']

In [105]:
tickers[0]

'AABA'

In [82]:
len(os.listdir(data_path_attn))

1738

In [19]:
data_path_attn

'/home/kikyo/data/qt/attention_data_NYSE/'

In [10]:
np.max(hpg[1])

4066

In [3]:
# generate rr data
for index, ticker in enumerate(tqdm(tickers)):
    single_EOD0 = np.genfromtxt(os.path.join(data_path_eod, market + '_' + ticker + '_30Y.csv'), dtype=str, delimiter=',', skip_header=True)
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
    day_index = 0
    single_data = []
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
        if day_time >= begin_time and day + 30 < len(single_EOD0):
            data_dict = {}
            single_EOD = single_EOD0[:, 1:].astype(float) * 100
            days_closeP = [single_EOD[day][3], single_EOD[day - 1][3], single_EOD[day - 5][3], 
                         single_EOD[day - 10][3], single_EOD[day - 20][3], single_EOD[day - 30][3], 
                         single_EOD[day + 1][3], single_EOD[day + 5][3], single_EOD[day + 30][3], 
                         single_EOD[day][-1]]
            data_dict['rr'] = (days_closeP[0] - days_closeP[1]) / days_closeP[1] if days_closeP[1] != 0 else -1234
            data_dict['rr5'] = (days_closeP[0] - days_closeP[2]) / days_closeP[2] if days_closeP[2] != 0 else -1234
            data_dict['rr10'] = (days_closeP[0] - days_closeP[3]) / days_closeP[3] if days_closeP[3] != 0 else -1234
            data_dict['rr20'] = (days_closeP[0] - days_closeP[4]) / days_closeP[4] if days_closeP[4] != 0 else -1234
            data_dict['rr30'] = (days_closeP[0] - days_closeP[5]) / days_closeP[5] if days_closeP[5] != 0 else -1234
            data_dict['rr-1'] = (days_closeP[6] - days_closeP[0]) / days_closeP[0] if days_closeP[6] != 0 else -1234
            data_dict['rr-5'] = (days_closeP[7] - days_closeP[0]) / days_closeP[0] if days_closeP[7] != 0 else -1234
            data_dict['rr-30'] = (days_closeP[8] - days_closeP[0]) / days_closeP[0] if days_closeP[8] != 0 else -1234
            data_dict['volumn'] = days_closeP[9] / 100
            single_data.append(data_dict)
    single_data = pd.DataFrame(single_data)
    break
#     single_data.to_csv(data_path_rr + market + ticker + '.csv')

  0%|          | 0/1026 [00:00<?, ?it/s]

In [70]:
ticker

'AABA'

In [69]:
days_closeP

[65.57, 65.4, 67.45, 67.91, 65.58, 66.08, 67.52, 70.51, 71.25, 325358215.5]

In [71]:
stock_path = [data_path_rr + npy_path for npy_path in os.listdir(data_path_rr)]

In [72]:
stock_path

['/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_PUI.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_MCHP.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_FWRD.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_EXLS.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_URBN.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_MPWR.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_SVVC.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_MNTA.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_UGLD.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/.ipynb_checkpoints',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_HCOM.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_PRSC.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_GPOR.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_CONN.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_SPLK.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_ZEUS.csv',
 '/home/kikyo/data/qt/rr8_volumn_NASDAQ/NASDAQ_USAP.csv',
 '/home/kiky

In [73]:
len(stock_path)

1027

In [74]:
del stock_path[9]

In [75]:
for rr_data in tqdm(stock_path):
    single_data = pd.read_csv(rr_data, index_col=0)
    if len(single_data) < 1244:
        row = np.full((1244 - single_data.shape[0], 10), -1234)
        index = range(single_data.shape[0], 1244)
        pd_1234 = pd.DataFrame(row, index=index, columns=col)
        single_data = pd.concat([single_data, pd_1234])
    elif len(single_data) > 1244:
        single_data = single_data.drop(index=range(1244, single_data.shape[0]))
    single_data.to_csv(rr_data)

  0%|          | 0/1026 [00:00<?, ?it/s]

In [53]:
np.min(single_data.values[:, -1])

70043.0

In [95]:
single_data['rr']

0       0.539871
1       0.537693
2       0.501949
3       0.537202
4       0.505111
          ...   
1239    0.532932
1240    0.459857
1241    0.536964
1242    0.382970
1243    0.558297
Name: rr, Length: 1244, dtype: float64

In [96]:
single_data['rr'] = (single_data['rr'] - np.min(single_data['rr'])) / (np.max(single_data['rr']) - np.min(single_data['rr']))

In [91]:
(single_data['rr'] - np.min(single_data['rr'])) / (np.max(single_data['rr']) - np.min(single_data['rr']))

0       0.539871
1       0.537693
2       0.501949
3       0.537202
4       0.505111
          ...   
1239    0.532932
1240    0.459857
1241    0.536964
1242    0.382970
1243    0.558297
Name: rr, Length: 1244, dtype: float64

In [ ]:
for index, ticker in enumerate(tqdm(tickers)):
    single_EOD0 = np.genfromtxt(os.path.join(data_path_eod, market + '_' + ticker + '_30Y.csv'), dtype=str, delimiter=',', skip_header=True)
    single_data = []
    for day in range(len(single_EOD0)):
        day_time = datetime.strptime(single_EOD0[day][0].split(' ')[0] + ' 00:00:00', date_format)
        if day_time >= begin_time and day + 30 < len(single_EOD0):
            data_dict = {}
            single_EOD = single_EOD0[:, 1:].astype(np.float64)
            days_weight = single_EOD[day][0] * single_EOD[day][4]
            data_dict['hyper_weight'] = days_weight if days_weight != 0 else -1234
            single_data.append(data_dict)
    single_data = pd.DataFrame(single_data)
    if len(single_data) < 1244:
        row = np.full((1244 - single_data.shape[0], 1), -1234)
        index = range(single_data.shape[0], 1244)
        pd_1234 = pd.DataFrame(row, index=index, columns=['hyper_weight'])
        single_data = pd.concat([single_data, pd_1234])
    elif len(single_data) > 1244:
        single_data = single_data.drop(index=range(1244, single_data.shape[0]))
    single_data.to_csv(data_path_attn + market + '/' + ticker + '.csv')

  2%|▏         | 35/1737 [04:35<4:51:32, 10.28s/it]

In [87]:
weight_path = '/home/kikyo/data/qt/ConceptWeightList/'
npy_path_list = os.listdir(weight_path)
npy_path_list.sort(key=lambda x:int(x.split('.')[0]))
npy_list = [np.load(weight_path + npy_path) for npy_path in npy_path_list]
col = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'rr-1', 'rr-5', 'rr-30', 'volumn']
weight_npy = np.concatenate(npy_list)

ValueError: invalid literal for int() with base 10: ''

In [46]:
np.save('/home/kikyo/data/qt/PreConceptWeight_' + market + '.npy', weight_npy)

In [78]:
col = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'rr-1', 'rr-5', 'rr-30', 'volumn']
data_path_rr = '/home/kikyo/data/qt/rr8_volumn_NASDAQ/'

In [61]:
col = ['rr', 'rr5', 'rr10', 'rr20', 'rr30', 'rr-1', 'rr-5', 'rr-30', 'volumn']
rr_data = '/home/kikyo/data/qt/rr8_volumn_NYSE/NYSE_ABBV.csv'
single_data = pd.read_csv(rr_data, index_col=0)